In [1347]:
import os
import polars as pl
pl.Config(tbl_rows=500)

In [1348]:
aut = pl.read_parquet(os.path.join("data","aut_vyber.parquet"))

In [1349]:
wikid = pl.read_parquet(os.path.join("data","wikidata.parquet"))

## Personální autority

In [1351]:
aut.sample(10)

024_2,024_a,046_f,046_g,100_7,100_a,100_d,100_ind1,110_a,370_a,370_b,370_c,370_f,372_a,373_a,374_a,375_a,377_a,400_a,400_d,400_i,400_ind1,410_a,411_a,430_a,450_a,500_a,500_i,500_ind1,550_7,678_a,856_u,leader,001
list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],str,str
null,null,"[""1979""]",null,"[""mzk2007295177""]","[""Šudoma, Milan,""]","[""1979-""]","[""1""]",null,null,null,"[""Česko""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"[""Narozen 1979. Anglista, pedagog, autor učebnic a metodických materiálů pro výuku angličtiny.""]",null,""" cz a22 n 4500""","""mzk2007295177"""
null,null,"[""1923""]","[""2000""]","[""xx0224036""]","[""Myklebost, Hallstein,""]","[""1923-2000""]","[""1""]",null,null,null,"[""Norsko""]",null,"[""geografie"", ""urbanizace""]",null,"[""geografové"", ""vysokoškolští učitelé""]","[""muž""]","[""nor""]",null,null,null,null,null,null,null,null,null,null,null,null,"[""Norský geograf a vysokoškolský učitel zaměřený na výzkum urbanizace.""]",null,""" cz a22 n 4500""","""xx0224036"""
null,null,null,null,"[""mzk2012704033""]","[""Hartmann, Karl-Theo""]",null,"[""1""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"[""Německý autor počítačových příruček.""]",null,""" nz a22 n 4500""","""mzk2012704033"""
null,null,null,null,"[""mub2016917019""]","[""Schönbrunn, Sarah""]",null,"[""1""]",null,null,null,"[""Německo""]",null,"[""němčina"", ""lingvistika"", … ""jazyková výuka""]",null,"[""lingvistky"", ""germanistky""]","[""žena""]","[""ger""]","[""Schonbrunn, Sarah""]",null,null,"[""1""]",null,null,null,null,null,null,null,null,"[""Německá lingvistka, germanistka, specialistka na pedagogický výzkum v jazykové oblasti.""]",null,""" nz a22 n 4500""","""mub2016917019"""
null,null,null,null,"[""xx0218867""]","[""Mosse, Ramona""]",null,"[""1""]",null,null,null,null,null,"[""filozofie"", ""divadlo"", … ""kulturní politika""]","[""Freie Universität Berlin"", ""Johns Hopkins University""]","[""teatroložky"", ""vysokoškolské učitelky""]","[""žena""]","[""eng"", ""ger""]","[""Thomasius, Ramona""]",null,null,"[""1""]",null,null,null,null,null,null,null,null,"[""Teatroložka a vysokoškolská pedagožka, zaměřená na vztahy mezi filozofií a divadlem, teorii tragédie, divadlo moderny, anglo-americké divadlo, operu a hudební divadlo současnosti, výkonnost a teatrálnost, etiku a estetiku.""]",null,""" cz a22 n 4500""","""xx0218867"""
null,null,"[""1950""]",null,"[""jo2016927689""]","[""Cotkin, George,""]","[""1950-""]","[""1""]",null,null,null,"[""Spojené státy americké""]",null,"[""dějiny"", ""kultura""]",null,"[""vysokoškolští učitelé""]","[""muž""]","[""eng""]",null,null,null,null,null,null,null,null,null,null,null,null,"[""Profesor americké historie a kultury a intelektuální historie, též zaměřený na průnik morálky a historie.""]",null,""" nz a22 n 4500""","""jo2016927689"""
null,null,null,null,"[""jo20181012364""]","[""Verjans, Pierre""]",null,"[""1""]",null,null,null,"[""Belgie""]",null,"[""politologie""]",null,"[""politologové""]","[""muž""]","[""fre""]",null,null,null,null,null,null,null,null,null,null,null,null,"[""Belgický profesor politologie.""]",null,""" nz a22 n 4500""","""jo20181012364"""
null,null,null,null,null,null,null,null,null,null,null,"[""Rusko""]",null,"[""filologie"", ""lexikologie"", … ""jazykové vlivy""]",null,null,null,"[""rus""]",null,null,null,null,null,"[""Finno-ugorskije jazyki narodov Rossii v uslovijach vzaimodejstvija s jazykami raznych sistem (sympozium)""]",null,null,null,null,null,null,null,null,""" nz a22 n 4500""","""ko20191047644"""
null,null,null,null,"[""xx0124631""]","[""Lash, Batton""]",null,"[""1""]",null,null,nu

In [1352]:
aut = aut.explode("375_a").with_columns(pl.when(pl.col("375_a") == "muž").then(pl.lit("m")).when(pl.col("375_a") == "žena").then(pl.lit("f")).otherwise(None)
    .alias("gender"))

In [1353]:
def rok_z_autorit(retezec):
    try:
        rok = int(re.search(r"\b\d{1,4}(?!\.)\b", retezec).group())
        if "př. Kr." in retezec:
            return -rok
        else:
            return rok
    except:
        return None

def roky_z_autorit(retezec):
    if "činn" in retezec:
        return {"narozeni": None, "umrti": None}
    narozeni = rok_z_autorit(retezec.split('-')[0])
    umrti = rok_z_autorit(retezec.split('-')[1])
    if umrti and narozeni:
        if (umrti < 0) and (narozeni > 0):
            narozeni = -narozeni
    return {"narozeni": narozeni, "umrti": umrti}

In [1354]:
roky_z_autorit("43 př. Kr.-asi 18 po Kr.")

{'narozeni': -43, 'umrti': 18}

In [1355]:
roky_z_autorit("129-asi 216")

{'narozeni': 129, 'umrti': 216}

In [1356]:
roky_z_autorit("činný 2488-2470 př. Kr.")

{'narozeni': None, 'umrti': None}

In [1357]:
roky_z_autorit("činný 16. století-17. století")

{'narozeni': None, 'umrti': None}

In [1358]:
roky_z_autorit("činná 1875-1876")

{'narozeni': None, 'umrti': None}

In [1359]:
aut = aut.explode("100_d").with_columns(pl.col("100_d").map_elements(roky_z_autorit).alias("letopocty")).unnest('letopocty')

C:\Users\micha\AppData\Local\Temp\ipykernel_20692\3319942890.py:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  aut = aut.explode("100_d").with_columns(pl.col("100_d").map_elements(roky_z_autorit).alias("letopocty")).unnest('letopocty')


In [1360]:
aut_vyplnene

100_7,narozeni,umrti,gender
str,i64,i64,str
"""jk01010388""",1903,1942,null
"""jk01010784""",1963,null,null
"""jk01010820""",null,1979,null
"""jk01010960""",1380,1445,null
"""jk01010979""",1920,null,null
"""jk01011132""",1928,null,null
"""jk01011169""",1950,null,null
"""jk01011741""",1875,1929,null
"""jk01012259""",1941,null,null


In [1361]:
str(aut_vyplnene.head(10))

'shape: (10, 4)\n┌────────────┬──────────┬───────┬────────┐\n│ 100_7      ┆ narozeni ┆ umrti ┆ gender │\n│ ---        ┆ ---      ┆ ---   ┆ ---    │\n│ str        ┆ i64      ┆ i64   ┆ str    │\n╞════════════╪══════════╪═══════╪════════╡\n│ jk01010388 ┆ 1903     ┆ 1942  ┆ null   │\n│ jk01010784 ┆ 1963     ┆ null  ┆ null   │\n│ jk01010820 ┆ null     ┆ 1979  ┆ null   │\n│ jk01010960 ┆ 1380     ┆ 1445  ┆ null   │\n│ jk01010979 ┆ 1920     ┆ null  ┆ null   │\n│ jk01011132 ┆ 1928     ┆ null  ┆ null   │\n│ jk01011169 ┆ 1950     ┆ null  ┆ null   │\n│ jk01011741 ┆ 1875     ┆ 1929  ┆ null   │\n│ jk01012259 ┆ 1941     ┆ null  ┆ null   │\n│ jk01012814 ┆ 1927     ┆ null  ┆ null   │\n└────────────┴──────────┴───────┴────────┘'

### Doplnění

In [1363]:
aut = aut.explode('678_a').with_columns(pl.col('678_a').map_elements(lambda x: x.split(' ')[0] if x else None).alias('bio1'))

C:\Users\micha\AppData\Local\Temp\ipykernel_20692\1683128886.py:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  aut = aut.explode('678_a').with_columns(pl.col('678_a').map_elements(lambda x: x.split(' ')[0] if x else None).alias('bio1'))


In [1364]:
muzska_bio1 = aut.filter(pl.col('gender') == 'm').group_by(['bio1']).len().sort(by='len',descending=True).head(500)
muzska_bio1

bio1,len
str,u32
"""Narozen""",36618
"""Americký""",31853
"""Německý""",31557
"""Britský""",12799
"""Francouzský""",11670
"""Polský""",11649
"""Ruský""",11152
"""Italský""",7567
"""Slovenský""",4605


In [1365]:
muzska_slova = muzska_bio1.select(pl.col('bio1')).to_series().to_list()
muzska_slova = [m for m in muzska_slova if m]
muzska_slova = [m for m in muzska_slova if len(m) > 1]
muzska_slova = [m for m in muzska_slova if m[-1] != 'á']
muzska_slova = [m for m in muzska_slova if m[-1] not in ['.','í'] and m[-2] != '.' and m != 'MD,' and m != 'MB' and m != 'Původem'
               and m != 'Narozena' and m != 'Americká' and m != 'Pseudonym' and m != 'Publikace' and m != 'PhD,' and m != 'DM,']
muzska_slova    

['Narozen',
 'Americký',
 'Německý',
 'Britský',
 'Francouzský',
 'Polský',
 'Ruský',
 'Italský',
 'Slovenský',
 'Rakouský',
 'Španělský',
 'Vysokoškolský',
 'Autor',
 'Nizozemský',
 'Ukrajinský',
 'Švýcarský',
 'Lékař,',
 'Kanadský',
 'Srbský',
 'Maďarský',
 'Belgický',
 'Švédský',
 'Odborník',
 'Chorvatský',
 'Australský',
 'Anglický',
 'Profesor',
 'Bulharský',
 'Japonský',
 'Indický',
 'Slovinský',
 'Čínský',
 'Rumunský',
 'Dánský',
 'Hudebník,',
 'Finský',
 'Běloruský',
 'Izraelský',
 'Historik',
 'Norský',
 'Portugalský',
 'Historik,',
 'Brazilský',
 'Irský',
 'Sovětský',
 'Řecký',
 'Překladatel',
 'Spoluautor',
 'Počítačový',
 'Argentinský',
 'Mexický',
 'Makedonský',
 'Právník,',
 'Skotský',
 'Katolický',
 'Zemřel',
 'Ekonom,',
 'Narodil',
 'Spisovatel,',
 'Chemik,',
 'Výzkumný',
 'Novinář,',
 'Černohorský',
 'Litevský',
 'Politolog,',
 'Novozélandský',
 'Lékař;',
 'Turecký',
 'Bosenský',
 'Fyzik,',
 'Kolumbijský',
 'Právník',
 'Tiskař',
 'Inženýr,',
 'Matematik,',
 'Rakousko-u

In [1366]:
[m for m in muzska_slova if m[-1] == 'a']

['Lingvista',
 'Specialista',
 'Publicista',
 'Religionista',
 'Germanista',
 'Starosta',
 'Předseda',
 'Urbanista']

In [1367]:
zenska_bio1 = aut.filter(pl.col('gender') == 'f').group_by(['bio1']).len().sort(by='len',descending=True).head(500)
zenska_bio1

bio1,len
str,u32
"""Narozena""",17909
"""Americká""",13322
"""Německá""",7550
"""Britská""",5924
"""MUDr.,""",5427
"""Polská""",4907
"""Mgr.,""",4705
"""Francouzská""",3527
"""Ruská""",3047


In [1368]:
zenska_slova = zenska_bio1.select(pl.col('bio1')).to_series().to_list()
zenska_slova = [m for m in zenska_slova if m]
zenska_slova = [m for m in zenska_slova if len(m) > 1]
zenska_slova = [m for m in zenska_slova if m[-1] not in ['.','í'] and m[-2] != '.' and m != 'MD,' and m != 'MB' and m != 'Původem' and m != 'Narozen' and m != 'Pseudonym' and m != 'Publikace' and m != 'PhD,' and m != 'Mgr,' and m[-1] != 'y' and m[-1] != 'ý' and m != 'Ing,' and m != 'MA,' and m != 'MUDr,' and m != 'MBA,']
zenska_slova    

['Narozena',
 'Americká',
 'Německá',
 'Britská',
 'Polská',
 'Francouzská',
 'Ruská',
 'Slovenská',
 'Italská',
 'Autorka',
 'Vysokoškolská',
 'Rakouská',
 'Kanadská',
 'Španělská',
 'Australská',
 'Překladatelka',
 'Historička',
 'Ukrajinská',
 'Nizozemská',
 'Srbská',
 'Chorvatská',
 'Švédská',
 'Anglická',
 'Odbornice',
 'Švýcarská',
 'Maďarská',
 'Profesorka',
 'Bulharská',
 'Belgická',
 'Slovinská',
 'Finská',
 'Rumunská',
 'Učitelka',
 'Dánská',
 'Portugalská',
 'Historička,',
 'Spoluautorka',
 'Irská',
 'Lékařka,',
 'Japonská',
 'Norská',
 'Indická',
 'Izraelská',
 'Čínská',
 'Běloruská',
 'Ilustrátorka',
 'Novinářka,',
 'Spisovatelka,',
 'Brazilská',
 'Spisovatelka',
 'Lektorka',
 'Právnička,',
 'Makedonská',
 'Argentinská',
 'Pedagožka,',
 'Mexická',
 'Psycholožka,',
 'Výzkumná',
 'Filoložka,',
 'Řecká',
 'Hudebnice,',
 'Studentka',
 'Socioložka,',
 'Redaktorka',
 'Lingvistka,',
 'Ekonomka,',
 'Novozélandská',
 'Politoložka,',
 'Ilustrátorka,',
 'Architektka,',
 'Novinářka',


In [1369]:
aut.filter((pl.col('gender') == 'm') & (pl.col('bio1') == 'Narozena'))

024_2,024_a,046_f,046_g,100_7,100_a,100_d,100_ind1,110_a,370_a,370_b,370_c,370_f,372_a,373_a,374_a,375_a,377_a,400_a,400_d,400_i,400_ind1,410_a,411_a,430_a,450_a,500_a,500_i,500_ind1,550_7,678_a,856_u,leader,001,gender,narozeni,umrti,bio1
list[str],list[str],list[str],list[str],list[str],list[str],str,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],str,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],str,list[str],str,str,str,i64,i64,str
null,null,"[""1936""]",null,"[""jk01071028""]","[""Kurzová, Helena,""]","""1936-""","[""1""]",null,null,null,"[""Česko""]",null,"[""klasická filologie"", ""překlady z latiny""]",null,"[""klasické filoložky"", ""překladatelky"", ""vědkyně""]","""muž""","[""cze""]",null,null,null,null,null,null,null,null,null,null,null,null,"""Narozena 1936. Klasická filolo…",null,""" cz a22 n 4500""","""jk01071028""","""m""",1936,null,"""Narozena"""
null,null,"[""1921""]","[""2014""]","[""jk01120331""]","[""Sojková, Zdenka,""]","""1921-2014""","[""1""]",null,"[""Klobuky, Česko""]",null,"[""Česko""]",null,null,null,"[""spisovatelky"", ""literární historičky"", … ""středoškolské učitelky""]","""muž""","[""cze""]",null,null,null,null,null,null,null,null,null,null,null,null,"""Narozena 8. 12. 1921 v Klobuká…","[""https://cs.wikipedia.org/wiki/Zdenka_Sojkov%C3%A1""]",""" cz a22 n 4500""","""jk01120331""","""m""",1921,2014,"""Narozena"""
"[""isni"", ""wikidata""]","[""0000000058062328"", ""Q10853464""]","[""1941""]","[""2022""]","[""jk01141269""]","[""Valentová, Anna,""]","""1941-2022""","[""1""]",null,"[""Lipník nad Bečvou, Česko""]",null,"[""Česko""]",null,"[""maďarská literatura""]",null,"[""překladatelky"", ""redaktorky"", ""spisovatelky""]","""muž""","[""cze"", ""hun"", ""eng""]",null,null,null,null,null,null,null,null,null,null,null,null,"""Narozena 16. 4. 1941 v Lipníku…",null,""" cz a22 n 4500""","""jk01141269""","""m""",1941,2022,"""Narozena"""
null,null,"[""1954""]",null,"[""jn19981000894""]","[""Dvořáková, Eva,""]","""1954 prosinec 30.-""","[""1""]",null,null,null,"[""Česko""]",null,"[""fotografování"", ""reklamní fotografie""]",null,"[""fotografky""]","""muž""",null,null,null,null,null,null,null,null,null,null,null,null,null,"""Narozena 30.12.1954. Fotografk…",null,""" cz a22 n 4500""","""jn19981000894""","""m""",1954,null,"""Narozena"""
"[""isni"", ""orcid"", ""wikidata""]","[""0000000138567115"", ""0000-0003-0494-2620"", ""Q30830691""]","[""1957""]",null,"[""nlk20000084855""]","[""Foretová, Lenka,""]","""1957-""","[""1""]",null,null,null,"[""Česko""]",null,null,null,"[""lékařky""]","""muž""","[""cze""]",null,null,null,null,null,null,null,null,null,null,null,null,"""Narozena roku 1957. Lékařka, z…",null,""" cz a22 n 4500""","""nlk20000084855""","""m""",1957,null,"""Narozena"""
null,null,"[""1958""]",null,"[""jn20010309510""]","[""Navrátilová, Danuše,""]","""1958-""","[""1""]",null,"[null, ""Praha, Česko""]",null,"[""Česko"", ""Česko""]",null,"[""překladatelství"", ""překlady z francouzštiny""]",null,"[""překladatelky""]","""muž""","[""cze""]",null,null,null,null,null,null,null,null,null,null,null,null,"""Narozena 11. 1. 1958 v Praze. …",null,""" cz a22 n 4500""","""jn20010309510""","""m""",1958,null,"""Narozena"""
"[""wikidata""]","[""Q112374388""]","[""1951""]",null,"[""xx0007161""]","[""Cichrová, Kateřina,""]","""1951-""","[""1""]",null,"[""České Budějovice, Česko""]",null,"[""Česko""]","[""České Budějovice, Česko""]","[""etnografie"", ""dějiny umění"", … ""kurátorství sbírek""]","[""Národní památkový ústav. Územní památková správa v Českých Budějovicích"", ""Národní památkový ústav. Územní odborné pracoviště v Českých Budějovicích""]","[""historičky umění"", ""etnografky"", … ""kurátorky sbírek""]","""muž""","[""cze""]",null,null,null,null,null,null,null,null,null,null,null,null,"""Narozena 1951 v Českých Budějo…",null,""" cz a22 n 4500""","""xx0007161""","""m""",1951,null,"""Naro

To je asi dobrý důvod převážit původní označení genderu, že.

In [1371]:
aut = aut.explode('100_7')

In [1372]:
skoro_urcite_muzi = aut.filter(pl.col('bio1').is_in(muzska_slova))
predelat_na_muze = skoro_urcite_muzi.select(pl.col('100_7')).to_series().to_list()
skoro_urcite_muzi

024_2,024_a,046_f,046_g,100_7,100_a,100_d,100_ind1,110_a,370_a,370_b,370_c,370_f,372_a,373_a,374_a,375_a,377_a,400_a,400_d,400_i,400_ind1,410_a,411_a,430_a,450_a,500_a,500_i,500_ind1,550_7,678_a,856_u,leader,001,gender,narozeni,umrti,bio1
list[str],list[str],list[str],list[str],str,list[str],str,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],str,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],str,list[str],str,str,str,i64,i64,str
null,null,"[""1931""]",null,"""jk01010001""","[""Abraham, Jiří,""]","""1931-""","[""1""]",null,null,null,"[""Česko""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Narozen 14.12.1931 v Osvračíně…",null,""" nz a22 n 4500""","""jk01010001""",null,1931,null,"""Narozen"""
null,null,"[""1890""]","[""1933""]","""jk01010002""","[""Abraham, Josef,""]","""1890-1933""","[""1""]",null,null,null,"[""Česko""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Narozen 19.11.1890 v Osvračíně…",null,""" nz a22 n 4500""","""jk01010002""",null,1890,1933,"""Narozen"""
null,null,"[""1957""]",null,"""jk01010003""","[""Abraham, Pavel,""]","""1957-""","[""1""]",null,null,null,"[""Česko""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Narozen roku 1957. Student fil…",null,""" nz a22 n 4500""","""jk01010003""",null,1957,null,"""Narozen"""
null,null,"[""1889""]","[""1963""]","""jk01010004""","[""Absolon, Josef,""]","""1889-1963""","[""1""]",null,"[""Odrlice, Senice na Hané, Česko""]",null,"[""Česko""]",null,null,null,"[""katoličtí kněží"", ""básníci""]","""muž""",null,null,null,null,null,null,null,null,null,null,null,null,null,"""Narozen 27. 1. 1889 v Odrlicíc…",null,""" cz a22 n 4500""","""jk01010004""","""m""",1889,1963,"""Narozen"""
"[""isni"", ""wikidata""]","[""0000000109542912"", ""Q1063371""]","[""1877""]","[""1960""]","""jk01010005""","[""Absolon, Karel,""]","""1877-1960""","[""1""]",null,null,null,"[""Česko""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Narozen 16.6.1877 v Boskovicíc…",null,""" cz a22 n 4500""","""jk01010005""",null,1877,1960,"""Narozen"""
null,null,"[""1843""]","[""1882""]","""jk01010009""","[""Absolon, Vilibald,""]","""1843-1882""","[""1""]",null,null,null,"[""Česko""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Narozen roku 1843 ve Fryštáku,…",null,""" nz a22 n 4500""","""jk01010009""",null,1843,1882,"""Narozen"""
null,null,"[""1803""]","[""1875""]","""jk01010012""","[""Ackermann, Josef,""]","""1803-1875""","[""1""]",null,null,null,"[""Česko""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Narozen 8.3.1803 v Dobřanech, …",null,""" nz a22 n 4500""","""jk01010012""",null,1803,1875,"""Narozen"""
null,null,"[""1832""]","[""1880""]","""jk01010013""","[""Ackermann, Josef,""]","""1832-1880""","[""1""]",null,null,null,"[""Česko""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Narozen roku 1832, zemřel roku…",null,""" nz a22 n 4500""","""jk01010013""",null,1832,1880,"""Narozen"""
null,null,null,null,"""jk01010016""","[""Acler, Jar.""]",null,"[""1""]",null,null,null,"[""Česko""]",null,"[""překladatelství""]",null,"[""překladatelé""]","""muž""","[""cze"", ""eng""]","[""Acler, Jaromír"", ""Acler, Jaroslav""]",null,null,"[""1"", ""1""]",null,null,null,null,null,null,null,null,"""Překladatel dobrodružných a de…",null,""" cz a22 n 4500""","""jk01010016""","""m""",null,null,"""Překladatel"""


In [1373]:
skoro_urcite_zeny = aut.filter(pl.col('bio1').is_in(zenska_slova))
predelat_na_zeny = skoro_urcite_zeny.select(pl.col('100_7')).to_series().to_list()
skoro_urcite_zeny

024_2,024_a,046_f,046_g,100_7,100_a,100_d,100_ind1,110_a,370_a,370_b,370_c,370_f,372_a,373_a,374_a,375_a,377_a,400_a,400_d,400_i,400_ind1,410_a,411_a,430_a,450_a,500_a,500_i,500_ind1,550_7,678_a,856_u,leader,001,gender,narozeni,umrti,bio1
list[str],list[str],list[str],list[str],str,list[str],str,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],str,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],str,list[str],str,str,str,i64,i64,str
null,null,"[""1904""]","[""1995""]","""jk01010010""","[""Abžoltovská-Klichová, Vlastimila,""]","""1904-1995""","[""1""]",null,null,null,"[""Česko""]",null,null,null,null,null,null,"[""Abžoltovská, Vlastimila,""]","[""1904-1995""]","[""Rodné jméno:""]","[""1""]",null,null,null,null,null,null,null,null,"""Narozena 1.10.1904 v Olšance n…",null,""" cz a22 n 4500""","""jk01010010""",null,1904,1995,"""Narozena"""
null,null,null,null,"""jk01010014""","[""Ackermannová, Marie""]",null,"[""1""]",null,null,null,"[""Česko""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Autorka příručky o ručním tkan…",null,""" cz a22 n 4500""","""jk01010014""",null,null,null,"""Autorka"""
null,null,"[""1905""]",null,"""jk01010032""","[""Adamcová, Marie,""]","""1905-""","[""1""]",null,null,null,"[""Česko""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Narozena 5.5.1905 ve Skramníká…",null,""" nz a22 n 4500""","""jk01010032""",null,1905,null,"""Narozena"""
null,null,"[""1935""]","[""1993""]","""jk01010031""","[""Adamcová, Alena,""]","""1935-1993""","[""1""]",null,null,null,"[""Česko""]",null,null,null,null,null,null,"[""Adamcová-Tokošová, Alena,""]","[""1935-1993""]",null,"[""1""]",null,null,null,null,null,null,null,null,"""Narozena 14. 6. 1935 v Praze, …",null,""" cz a22 n 4500""","""jk01010031""",null,1935,1993,"""Narozena"""
null,null,null,null,"""jk01010096""","[""Adamičková, Bedřiška""]",null,"[""1""]",null,null,null,"[""Česko""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Editorka prací o hudbě.""",null,""" cz a22 n 4500""","""jk01010096""",null,null,null,"""Editorka"""
null,null,"[""1921""]","[""2014""]","""jk01010104""","[""Adamírová, Jiřina,""]","""1921-2014""","[""1""]",null,"[""Praha, Česko""]",null,"[""Česko""]",null,"[""zdravotní tělesná výchova""]",null,"[""vysokoškolské učitelky""]","""žena""","[""cze""]",null,null,null,null,null,null,null,null,null,null,null,null,"""Narozena 30. 7. 1921 v Praze, …",null,""" cz a22 n 4500""","""jk01010104""","""f""",1921,2014,"""Narozena"""
null,null,null,null,"""jk01010108""","[""Adamová, Lenka""]",null,"[""1""]",null,null,null,"[""Česko""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Autorka práce o Ivanu Olbracht…",null,""" cz a22 n 4500""","""jk01010108""",null,null,null,"""Autorka"""
null,null,"[""1922""]",null,"""jk01010109""","[""Adamová, Olga,""]","""1922-""","[""1""]",null,null,null,"[""Česko""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Narozena 30.5.1922 v Praze. Ph…",null,""" nz a22 n 4500""","""jk01010109""",null,1922,null,"""Narozena"""
null,null,"[""1924""]","[""2001""]","""jk01010110""","[""Adamovská, Dagmar,""]","""1924-2001""","[""1""]",null,null,null,"[""Česko""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Narozena 6.7.1924 v Českých Bu…",null,""" cz a22 n 4500""","""jk01010110""",null,1924,2001,"""Narozena"""


In [1374]:
[x for x in zenska_slova if x in muzska_slova]

[]

In [1375]:
len(predelat_na_muze)

579208

In [1376]:
len(predelat_na_zeny)

198842

In [1377]:
predelat_na_zeny

['jk01010010',
 'jk01010014',
 'jk01010032',
 'jk01010031',
 'jk01010096',
 'jk01010104',
 'jk01010108',
 'jk01010109',
 'jk01010110',
 'jk01010106',
 'jk01010123',
 'jk01010122',
 'jk01010127',
 'jk01010158',
 'jk01010169',
 'jk01010170',
 'jk01010171',
 'jk01010189',
 'jk01010195',
 'jk01010196',
 'jk01010216',
 'jk01010238',
 'jk01010242',
 'jk01010250',
 'jk01010270',
 'jk01010284',
 'jk01010300',
 'jk01010315',
 'jk01010336',
 'jk01010337',
 'jk01010340',
 'jk01010338',
 'jk01010339',
 'jk01010341',
 'jk01010381',
 'jk01010386',
 'jk01010400',
 'jk01010403',
 'jk01010436',
 'jk01010437',
 'jk01010445',
 'jk01010451',
 'jk01010461',
 'jk01010466',
 'jk01010471',
 'jk01010475',
 'jk01010476',
 'jk01010485',
 'jk01010507',
 'jk01010537',
 'jk01010544',
 'jk01010572',
 'jk01010573',
 'jk01010576',
 'jk01010589',
 'jk01010606',
 'jk01010623',
 'jk01010614',
 'jk01010624',
 'jk01010625',
 'jk01010632',
 'jk01010663',
 'jk01010664',
 'jk01010660',
 'jk01010661',
 'jk01010662',
 'jk010106

In [1378]:
set(predelat_na_zeny).intersection(set(predelat_na_muze))

{None}

In [1379]:
predelat = predelat_na_muze + predelat_na_zeny

In [1380]:
aut_nebudeme_menit = aut.filter(~pl.col('100_7').is_in(predelat))
aut_budeme_menit = aut.filter(pl.col('100_7').is_in(predelat))

In [1381]:
aut_zmeneno = aut_budeme_menit.with_columns(pl.when(
    pl.col('100_7').is_in(predelat_na_muze)
).then(
    pl.lit('m')
).when(
    pl.col('100_7').is_in(predelat_na_zeny)
).then(pl.lit('f')
      ).otherwise(None).alias('gender'))

In [1382]:
aut_spojene = pl.concat([aut_nebudeme_menit, aut_zmeneno])

In [1383]:
aut_vyplnene = aut_spojene.filter(~pl.col("narozeni").is_null() | ~pl.col("umrti").is_null() | ~pl.col("umrti").is_null()).select(pl.col(['100_7','narozeni','umrti','gender']))

In [1384]:
len(aut_vyplnene)

653575

## Wikidata

In [1386]:
wikid.sample(10)

024_a,ceny,druh_umrti,facebook,instagram,jazykove_verze,label_cs,label_en,manzelstvo,obcanstvi,partnerstvo,popis_cs,popis_en,potomstvo,pricina_umrti,profese,role,skoly,sourozenectvo,strany,twitter,udalosti,vezeni,w_gender,w_misto_narozeni,w_misto_umrti,w_narozeni,w_narozeni_presne,w_umrti,w_umrti_presne,web,wiki_cs,wiki_en,__index_level_0__
str,list[str],str,str,str,list[str],str,str,list[str],list[str],list[str],str,str,list[str],str,list[str],list[str],list[str],list[str],list[str],str,list[str],list[str],str,list[str],list[str],f64,str,f64,str,str,str,str,str
"""Q12033414""","[""zasloužilý umělec""]",null,null,null,"[""cs""]","""Libuše Hynková""","""Libuše Hynková""",null,"[""Československo""]",null,"""česká choreografka""","""Czech choreographer""",null,null,"[""choreograf"", ""učitel"", ""tanečník""]",null,null,null,null,null,null,null,"""žena""","[""Teplice""]",null,1923.0,"""+1923-10-28T00:00:00Z""",2012.0,"""+2012-01-06T00:00:00Z""",null,"""https://cs.wikipedia.org/wiki/…",null,"""jk01043456"""
"""Q112536280""",null,null,null,null,[],"""Savvas Frangos""","""Savvas Frangos""",null,null,null,"""lékař, specializován v oboru n…",null,null,null,"[""lékař""]",null,null,null,null,null,null,null,"""muž""","[""Lemesos""]",null,1961.0,"""+1961-00-00T00:00:00Z""",null,null,null,null,null,"""xx0236673"""
"""Q7940790""",null,null,null,null,"[""ca"", ""cs"", … ""en""]","""Marie Červinková-Riegrová""","""Marie Červinková-Riegrová""","[""Václav Červinka""]",null,null,"""česká spisovatelka a libretist…","""Czech writer and librettist (1…",null,null,"[""spisovatel"", ""libretista"", … ""básník""]",null,null,"[""Bohuslav Rieger"", ""Libuše Bráfová""]",null,null,null,null,"""žena""","[""Praha""]","[""Praha""]",1854.0,"""+1854-08-09T00:00:00Z""",1895.0,"""+1895-01-19T00:00:00Z""",null,"""https://cs.wikipedia.org/wiki/…","""https://en.wikipedia.org/wiki/…","""jk01021755"""
"""Q112403206""",null,null,null,null,[],"""Nikolaj Konstantinovič Karataj…",null,null,null,null,"""ruský ekonom""",null,null,null,"[""ekonom""]",null,null,null,null,null,null,null,null,null,null,1899.0,"""+1899-00-00T00:00:00Z""",null,null,null,null,null,"""jx20061125017"""
"""Q16163514""",null,null,null,null,"[""cs""]","""Sedlec""",null,null,null,null,"""zámek v Karlových Varech""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""https://cs.wikipedia.org/wiki/…",null,"""kv2012711579"""
"""Q12036182""",null,null,null,null,"[""cs""]","""Maxmilián Hájek""","""Maxmilián Hájek""",null,"[""Československo""]",null,"""český dirigent, hudební sklada…","""Czech conductor, composer and …",null,null,"[""hudební skladatel"", ""dirigent""]",null,"[""Pražská konzervatoř""]",null,null,null,null,null,"""muž""","[""Praha""]","[""Liberec""]",1909.0,"""+1909-08-15T00:00:00Z""",1969.0,"""+1969-03-16T00:00:00Z""",null,"""https://cs.wikipedia.org/wiki/…",null,"""ola200204197"""
"""Q10605873""",null,null,"""Daniel.Docekal""","""danieldocekal""","[""cs""]","""Daniel Dočekal""","""Daniel Dočekal""",null,"[""Česko""]",null,"""český novinář""","""Czech publicist""",null,null,"[""spisovatel"", ""novinář""]",null,null,null,null,"""MedvidekPU""",null,null,"""muž""",null,null,1967.0,"""+1967-00-00T00:00:00Z""",null,null,null,"""https://cs.wikipedia.org/wiki/…",null,"""jx20090814016"""
"""Q85074555""",null,null,null,null,[],"""Gilles Alleaume""","""Gilles Alleaume""",null,"[""Francie""]",null,null,"""Jesuit theologian (1641-1706)""",null,null,"[""teolog"", ""překladatel"", ""preceptor""]",null,null,null,null,null,null,null,"""muž""","[""Saint-Malo""]","[""Paříž""]",1641.0,"""+1641-05-20T00:00:00Z""",1706.0,"""+1706-07-02T00:00:00Z""",null,null,null,"""xx0325560"""
"""Q4221345""",null,null,null,null,"[""ru""]","""Kirill Rjabov""",null,null,null,null,"""ruský spisovatel, autor románů""",null,null,null,"[""spisovatel""]",null,null,null,null,null,null,null,"""muž""","[""Petrohrad""]",null,1983.0,"""+1983-01-01T00:00:00Z""",null,null,null,null,null,"""js202111

In [1387]:
wikid = wikid.with_columns(pl.when(pl.col("w_gender") == "muž").then(pl.lit("m")).when(pl.col("w_gender") == "žena").then(pl.lit("f")).otherwise(None).alias("gender"))

In [1388]:
wikid_vyplnene = wikid.with_columns(pl.col("w_narozeni").cast(int).alias("narozeni")).with_columns(pl.col("w_umrti").cast(int).alias("umrti")).rename({'__index_level_0__':'100_7'}).select(pl.col(['100_7','narozeni','umrti','gender'])).filter(~pl.col("narozeni").is_null() | ~pl.col("umrti").is_null()| ~pl.col("gender").is_null())
wikid_vyplnene

100_7,narozeni,umrti,gender
str,i64,i64,str
"""jk01010005""",1877,1960,"""m"""
"""jk01010022""",1890,1971,"""m"""
"""jk01010023""",1928,2008,"""m"""
"""jk01010026""",1889,1974,"""m"""
"""jk01010030""",1546,1599,"""m"""
"""jk01010034""",1926,null,"""m"""
"""jk01010036""",1853,1913,"""m"""
"""jk01010040""",1866,1946,"""m"""
"""jk01010048""",1926,2007,"""m"""


## Spojení

In [1390]:
def specialized_join(df1: pl.DataFrame, df2: pl.DataFrame) -> pl.DataFrame:
    # Perform outer join on '100_7' column
    joined = df1.join(
        df2,
        on="100_7",
        how="outer",
        suffix="_df2"
    )
    
    # Coalesce the columns, prioritizing df1 values
    result = joined.with_columns([
        pl.coalesce("narozeni", "narozeni_df2").alias("narozeni"),
        pl.coalesce("umrti", "umrti_df2").alias("umrti")
    ])
    
    # Drop the duplicate columns from df2
    result = result.drop(["narozeni_df2", "umrti_df2"])
    
    return result

In [1391]:
specialized_join(aut_vyplnene, wikid_vyplnene)

C:\Users\micha\AppData\Local\Temp\ipykernel_20692\35769344.py:3: DeprecationWarning: Use of `how='outer'` should be replaced with `how='full'`.
  joined = df1.join(


100_7,narozeni,umrti,gender,100_7_df2,gender_df2
str,i64,i64,str,str,str
"""jk01010784""",1963,null,null,null,null
"""jk01010820""",null,1979,null,null,null
"""jk01010960""",1380,1445,null,"""jk01010960""","""m"""
"""jk01010979""",1920,null,null,null,null
"""jk01011169""",1950,null,null,null,null
"""jk01011741""",1875,1929,null,null,null
"""jk01012259""",1941,null,null,null,null
"""jk01012814""",1927,null,null,null,null
"""jk01012880""",null,1945,null,"""jk01012880""","""m"""


In [1392]:
def specialized_join(df1: pl.DataFrame, df2: pl.DataFrame) -> pl.DataFrame:
    # Perform outer join
    joined = df1.join(
        df2,
        on="100_7",
        how="outer",
        suffix="_df2"
    )
    
    # Coalesce all columns, including the ID column
    result = joined.with_columns([
        pl.coalesce("100_7", "100_7_df2").alias("100_7"),
        pl.coalesce("narozeni", "narozeni_df2").alias("narozeni"),
        pl.coalesce("umrti", "umrti_df2").alias("umrti"),
        pl.coalesce("gender", "gender_df2").alias("gender")
    ])
    
    # Drop the duplicate columns
    result = result.drop(["100_7_df2", "narozeni_df2", "umrti_df2","gender_df2"])
    
    return result

### Teď co bude přesnější

In [1394]:
specialized_join(aut_vyplnene, wikid_vyplnene).with_columns((pl.col("umrti") - pl.col("narozeni")).alias("doziti")).drop_nulls(subset="doziti").select(pl.col("doziti")).median()

C:\Users\micha\AppData\Local\Temp\ipykernel_20692\1570944117.py:3: DeprecationWarning: Use of `how='outer'` should be replaced with `how='full'`.
  joined = df1.join(


doziti
f64
73.0


In [1395]:
specialized_join(aut_vyplnene, wikid_vyplnene).with_columns((pl.col("umrti") - pl.col("narozeni")).alias("doziti")).drop_nulls(subset="doziti").filter(pl.col("doziti") > 110)

C:\Users\micha\AppData\Local\Temp\ipykernel_20692\1570944117.py:3: DeprecationWarning: Use of `how='outer'` should be replaced with `how='full'`.
  joined = df1.join(


100_7,narozeni,umrti,gender,doziti
str,i64,i64,str,i64
"""jn19981002426""",117,263,null,146
"""jn20000700997""",1832,1981,null,149
"""jn20000701383""",228,347,null,119
"""nlk20000078653""",1813,1970,null,157
"""nlk20000085066""",1707,1847,null,140
"""nlk20000085156""",1470,1824,null,354
"""nlk20000087689""",1664,1776,null,112
"""nlk20000088140""",1664,1776,null,112
"""nlk20000090813""",1843,1984,null,141


In [1396]:
specialized_join(wikid_vyplnene,aut_vyplnene).with_columns((pl.col("umrti") - pl.col("narozeni")).alias("doziti")).drop_nulls(subset="doziti").select(pl.col("doziti")).median()

C:\Users\micha\AppData\Local\Temp\ipykernel_20692\1570944117.py:3: DeprecationWarning: Use of `how='outer'` should be replaced with `how='full'`.
  joined = df1.join(


doziti
f64
73.0


In [1397]:
specialized_join(wikid_vyplnene, aut_vyplnene).with_columns((pl.col("umrti") - pl.col("narozeni")).alias("doziti")).drop_nulls(subset="doziti").filter(pl.col("doziti") > 110)

C:\Users\micha\AppData\Local\Temp\ipykernel_20692\1570944117.py:3: DeprecationWarning: Use of `how='outer'` should be replaced with `how='full'`.
  joined = df1.join(


100_7,narozeni,umrti,gender,doziti
str,i64,i64,str,i64
"""jn19981002426""",117,263,null,146
"""jn20000700997""",1832,1981,null,149
"""jn20000701383""",228,347,null,119
"""nlk20000078653""",1813,1970,null,157
"""nlk20000085066""",1707,1847,null,140
"""nlk20000085156""",1470,1824,null,354
"""nlk20000087689""",1664,1776,null,112
"""nlk20000088140""",1664,1776,null,112
"""nlk20000090813""",1843,1984,null,141


In [1398]:
specialized_join(wikid_vyplnene, aut_vyplnene).with_columns((pl.col("umrti") - pl.col("narozeni")).alias("doziti")).drop_nulls(subset="doziti").filter(pl.col("doziti") < 10)

C:\Users\micha\AppData\Local\Temp\ipykernel_20692\1570944117.py:3: DeprecationWarning: Use of `how='outer'` should be replaced with `how='full'`.
  joined = df1.join(


100_7,narozeni,umrti,gender,doziti
str,i64,i64,str,i64
"""jn19981228002""",525,456,"""m""",-69
"""jn19981000928""",384,322,"""m""",-62
"""jn19981001000""",480,406,"""m""",-74
"""jn19981001510""",484,425,"""m""",-59
"""jn19981001886""",100,25,"""m""",-75
"""jn19981001925""",43,17,"""m""",-26
"""jn19981002132""",1000,1000,"""f""",0
"""jn19992017219""",496,406,"""m""",-90
"""jn19981002215""",460,395,"""m""",-65


In [1399]:
df = specialized_join(aut_vyplnene, wikid_vyplnene)

C:\Users\micha\AppData\Local\Temp\ipykernel_20692\1570944117.py:3: DeprecationWarning: Use of `how='outer'` should be replaced with `how='full'`.
  joined = df1.join(


## Opravy a uložení

In [1414]:
aut.filter(pl.col('100_7') == 'vse2005279019')

024_2,024_a,046_f,046_g,100_7,100_a,100_d,100_ind1,110_a,370_a,370_b,370_c,370_f,372_a,373_a,374_a,375_a,377_a,400_a,400_d,400_i,400_ind1,410_a,411_a,430_a,450_a,500_a,500_i,500_ind1,550_7,678_a,856_u,leader,001,gender,narozeni,umrti,bio1
list[str],list[str],list[str],list[str],str,list[str],str,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],str,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],str,list[str],str,str,str,i64,i64,str
null,null,"[""1948""]","[""2011""]","""vse2005279019""","[""Hope, Jeremy,""]","""1948-2011""","[""1""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Auditor, poradce, odborník na …",null,""" cz a22 n 4500""","""vse2005279019""",null,1948,2011,"""Auditor,"""


In [1402]:
len(df.filter(pl.col('narozeni').is_null()))

16073

In [1403]:
len(df.filter(~pl.col('narozeni').is_null()))

650339

In [1404]:
len(df.filter(~pl.col('umrti').is_null()))

256448

In [1405]:
len(df.filter(pl.col('umrti').is_null()))

409964

In [1406]:
len(df.filter(~pl.col('gender').is_null()))

595031

In [1407]:
len(df.filter(pl.col('gender').is_null()))

71381

In [1408]:
df.group_by('100_7').len().sort(by='len',descending=True)

100_7,len
str,u32
"""xx0277072""",2
"""xx0201785""",2
"""jo20241235133""",2
"""mzk2016925795""",2
"""jo2004220387""",2
"""xx0250288""",2
"""pna2012678496""",2
"""jx20111201016""",2
"""jn19981228067""",2


In [1409]:
len(df)

666412

In [1410]:
df = df.unique(subset=['100_7'], keep='first')

In [1411]:
len(df)

666358

In [1412]:
df.filter(pl.col('gender').is_null()).sample(20)

100_7,narozeni,umrti,gender
str,i64,i64,str
"""nlk20010098451""",1965,null,null
"""nlk20000091363""",1848,1915,null
"""aun2016919811""",1841,null,null
"""aun2010591294""",1841,null,null
"""nlk20000088768""",1880,null,null
"""aun20181007427""",1822,null,null
"""xx0127143""",1946,null,null
"""aun2017944138""",1895,null,null
"""ola200209296""",1932,null,null


In [1416]:
df = df.with_columns(
    (pl.col("umrti") - pl.col("narozeni")).alias("doziti")
).with_columns(pl.when(~pl.col('doziti').is_between(15,115)).then(None).otherwise(pl.col('umrti')).alias('umrti'))

In [1418]:
df

100_7,narozeni,umrti,gender,doziti
str,i64,i64,str,i64
"""jo20211109096""",1916,1949,"""m""",33
"""xx0254282""",1892,1969,"""m""",77
"""jx20110407001""",1973,null,"""f""",null
"""jcu2011636347""",1943,null,"""f""",null
"""mub2013757182""",1894,1973,"""m""",79
"""mzk2004261314""",1931,null,"""m""",null
"""xx0102910""",1940,null,"""f""",null
"""xx0103210""",1944,2005,"""m""",61
"""mzk2002144922""",1945,2020,"""m""",75


In [1420]:
df.sample(100)

100_7,narozeni,umrti,gender,doziti
str,i64,i64,str,i64
"""hka20231178320""",1947,null,"""m""",null
"""xx0076270""",1910,1978,"""m""",68
"""jk01040963""",1898,1981,"""m""",83
"""jn19990009228""",1957,null,null,null
"""xx0302125""",1936,null,"""m""",null
"""jk01061260""",1905,1979,"""m""",74
"""js20051031022""",1986,null,"""m""",null
"""jn20000603052""",1854,1921,null,67
"""utb20191039404""",1949,null,"""m""",null


In [1422]:
df.drop('doziti').write_parquet(os.path.join('data','narozeni-umrti-gender.parquet'))